In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder
import pickle

# =========================================================
# 1. VERİ SETİNİN OLUŞTURULMASI
# =========================================================
# Bu bölümde gerçek futbolcu verilerini temsil edecek şekilde
# sentetik (rastgele fakat mantıklı aralıklara sahip) bir veri seti üretilmektedir.
print("1. Veri Seti Oluşturuluyor...")

data = {
    'overall_rating': np.random.randint(50, 95, 200),   # Oyuncunun genel performans puanı
    'age': np.random.randint(18, 38, 200),              # Oyuncunun yaşı
    'goals': np.random.randint(0, 40, 200),             # Sezonluk gol sayısı
    'assists': np.random.randint(0, 25, 200),           # Sezonluk asist sayısı
    'position': np.random.choice(
        ['Forvet', 'Orta Saha', 'Defans', 'Kaleci'], 200
    ),                                                   # Oyuncunun mevkisi
    'jersey_number': np.random.randint(1, 99, 200)      # Model için anlamsız değişken
}

df = pd.DataFrame(data)

# Piyasa değeri, futbolcu özelliklerine bağlı olacak şekilde
# belirli katsayılar kullanılarak hesaplanmaktadır.
# Ayrıca küçük bir rastgele gürültü (noise) eklenerek gerçekçiliği artırılmıştır.
df['market_value'] = 0.0

for index, row in df.iterrows():
    base_price = (
        (row['overall_rating'] * 1_000_000)
        - (row['age'] * 500_000)
        + (row['goals'] * 200_000)
    )

    # Forvet oyuncularına piyasa değerinde ek avantaj tanınır
    if row['position'] == 'Forvet':
        base_price += 5_000_000

    noise = np.random.randint(-2_000_000, 2_000_000)

    # Piyasa değerinin negatif olmaması için alt sınır uygulanır
    df.at[index, 'market_value'] = max(base_price + noise, 500_000)

# Oluşturulan veri seti dışa aktarılır
df.to_csv('futbolcu_verisi.csv', index=False)
print("✅ Veri seti oluşturuldu.")

# =========================================================
# 2. VERİ ÖN İŞLEME
# =========================================================
print("\n2. Veri Ön İşleme Yapılıyor...")

# Sayısal kolonlardaki eksik değerler ortalama ile doldurulur
df.fillna(df.mean(numeric_only=True), inplace=True)

# Kategorik değişken olan 'position', makine öğrenmesi modeline
# uygun hale getirilmek için sayısal forma dönüştürülür
le = LabelEncoder()
df['position_encoded'] = le.fit_transform(df['position'])

# Orijinal metinsel 'position' sütunu modelden çıkarılır
df_model = df.drop(columns=['position'])

# =========================================================
# 3. GERİYE DOĞRU ELEME (BACKWARD ELIMINATION)
# =========================================================
print("\n3. Backward Elimination (Gereksiz Veri Temizliği)...")

# Bağımsız değişkenler (X) ve bağımlı değişken (y) ayrıştırılır
X = df_model.drop(columns=['market_value'])
y = df_model['market_value']

# OLS (Ordinary Least Squares) modeli kurularak
# değişkenlerin istatistiksel anlamlılığı incelenir
X_ols = sm.add_constant(X)
ols_model = sm.OLS(y, X_ols).fit()

# Her bir değişken için p-value değerleri elde edilir
p_values = ols_model.pvalues

# Sabit terim (const) backward elimination sürecine dahil edilmez
if 'const' in p_values:
    p_values = p_values.drop('const')

# En yüksek p-value değeri kontrol edilir
max_p_value = p_values.max()

# Anlamlılık seviyesi (α = 0.05) üzerinde kalan değişken varsa
# veri setinden çıkarılır
if max_p_value > 0.05:
    removed_feature = p_values.idxmax()
    print(f"🚫 Gereksiz Veri Tespit Edildi: '{removed_feature}' (P-value: {max_p_value:.4f})")
    print(f"🗑️ '{removed_feature}' veri setinden atılıyor...")
    X = X.drop(columns=[removed_feature])
else:
    print("✅ Tüm veriler istatistiksel olarak anlamlı bulundu.")

# =========================================================
# 4. MODEL EĞİTİMİ
# =========================================================
print("\n4. Model Eğitiliyor...")

# Veri seti eğitim ve test olarak ayrılır
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Doğrusal Regresyon modeli eğitilir
reg_model = LinearRegression()
reg_model.fit(X_train, y_train)

# Test verisi üzerinde tahmin yapılır
y_pred = reg_model.predict(X_test)

# Model performansı değerlendirilir
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("📊 SONUÇLAR:")
print(f"   R² Skoru: {r2:.4f}")
print(f"   MAE: {mae:,.0f}")
print(f"   MSE: {mse:,.0f}")

# =========================================================
# 5. MODELİN KAYDEDİLMESİ
# =========================================================
# Eğitilen model, daha sonra kullanılmak üzere
# pickle formatında diske kaydedilir
with open('model.pkl', 'wb') as file:
    pickle.dump(reg_model, file)

print("\n💾 Model başarıyla 'model.pkl' olarak kaydedildi.")


1. Veri Seti Oluşturuluyor...
✅ Veri seti oluşturuldu.

2. Veri Ön İşleme Yapılıyor...

3. Backward Elimination (Gereksiz Veri Temizliği)...
🚫 Gereksiz Veri Tespit Edildi: 'jersey_number' (P-value: 0.7796)
🗑️ 'jersey_number' veri setinden atılıyor...

4. Model Eğitiliyor...
📊 SONUÇLAR:
   R² Skoru: 0.9749
   MAE: 2,152,690
   MSE: 6,471,707,621,499

💾 Model başarıyla 'model.pkl' olarak kaydedildi.
